In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[
                                    {'label':'All Sites', 'value':'All'},
                                    {'label':'CCAFS SLC 40', 'value':'CS40'},
                                    {'label':'KSC LC 39A', 'value':'KL39'},
                                    {'label':'VAFB SLC 4E', 'value':'vs4'}
                                ], 
                                value='ALL', 
                                placeholder='Select a Launch Site here',
                                searchable='True'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def pie(site):
    if site == 'ALL':
        ddf = pd.DataFrame(spacex_df['Launch Site'])
        fig = px.pie(ddf, values=['Class'], names=['Launch Site'], title='Success Counts for Launch Sites in US')

    else:
        filtered_df= spacex_df[spacex_df['Launch Site'] == site]
        filtered_df1 = pd.DataFrame(filtered_df['Launch Site'])
        fig = px.pie(filtered_df1, values=['Class Count'], names=['Class'], title='Succes count of '+site)

        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload-slider", component_property="value")])


def build_scatter(site,payload):
    low,high = (payload[0], payload[1])
    # filter your weights out here since you need to filter it whether all sites or an individual site is selected
    filtered_df2 = spacex_df[spacex_df['Payload Mass (kg)']].between(low,high) # fix this line as instructed above

    if site == 'ALL':
        fig = px.scatter(filtered_df2,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for All Sites')

    else:
        #now we can use our filtered payload weights to filter further by site in our else statement
        filtered_df3 = filtered_df2[filtered_df2['Launch Site'] == site]
        fig = px.scatter(filtered_df3, x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for' + site)
        # now we can return fig once at the end of our function since fig is what we want either way.
	    # our if else will produce a different fig for us based on the condition, but variable name is the same

    return fig

# Run the app

if __name__ == '__main__':
    app.run_server()
